How to authenticate to Google APIs?



In [3]:
var execSync = require('child_process').execSync;
try {
    require.resolve('googleapis');
    require.resolve('google-auth-library');
} catch (e) {
    execSync('npm install googleapis google-auth-library');
}


// Load client secrets from a local file.
var path = require('path');
var fs = require('fs');
var google = require('googleapis');
var googleAuth = require('google-auth-library');

// If modifying these scopes, delete your previously saved credentials
// at ~/.credentials/calendar-nodejs-quickstart.json
var SCOPES = ['https://www.googleapis.com/auth/calendar.readonly'];
var TOKEN_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, '.credentials');
var TOKEN_PATH = path.join(TOKEN_DIR, 'calendar-nodejs-quickstart.json');
var SECRET_PATH = path.join(TOKEN_DIR, 'client_secret.json');
var credentials = JSON.parse(fs.readFileSync(SECRET_PATH));


'/Users/briancullinan/jupytangular2/node_modules/google-auth-library/lib/auth/googleauth.js'

How to exchange token for oauth client?



In [4]:

/**
 * Store token to disk be used in later program executions.
 *
 * @param {Object} token The token to store to disk.
 */
function storeToken(token) {
  try {
    fs.mkdirSync(TOKEN_DIR);
  } catch (err) {
    if (err.code != 'EEXIST') {
      throw err;
    }
  }
  fs.writeFile(TOKEN_PATH, JSON.stringify(token));
  console.log('Token stored to ' + TOKEN_PATH);
}

/**
 * Create an OAuth2 client with the given credentials, and then execute the
 * given callback function.
 *
 * @param {Object} credentials The authorization client credentials.
 * @param {function} callback The callback to call with the authorized client.
 */
var runSeleniumCell;
var authorize = (scopes = SCOPES) => {
    var clientSecret = credentials.installed.client_secret;
    var clientId = credentials.installed.client_id;
    var redirectUrl = credentials.installed.redirect_uris[0];
    var auth = new googleAuth();
    var oauth2Client = new auth.OAuth2(clientId, clientSecret, redirectUrl);

    try {
        // Check if we have previously stored a token.
        var token = fs.readFileSync(TOKEN_PATH);
        oauth2Client.credentials = JSON.parse(token);
        return Promise.resolve(oauth2Client);
    } catch (e) {
        if(e.code !== 'ENOENT') {
            throw e;
        } else {
            var authUrl = oauth2Client.generateAuthUrl({
                    access_type: 'offline',
                    scope: scopes
                });
            return importer.interpretAll(['selenium cell'])
            .then(r => runSeleniumCell = r[0].runInNewContext())
            .then(() => runSeleniumCell('authorize google access'))
            .then(authorizeSelenium => authorizeSelenium(authUrl))
            .then(code => new Promise((resolve, reject) => {
                oauth2Client.getToken(code, (err, token) => {
                    if(err){
                        return reject(err);
                    } else {
                        return resolve(token);
                    }
                });
            }))
            .then(token => {
                oauth2Client.credentials = token;
                storeToken(token);
                return oauth2Client;
            })
        }
    }
};
module.exports = authorize;
authorize;




[Function: authorize]

How to use selenium to authorize Google access?



In [6]:
var importer = require('../Core');

var approveSelenium = (authUrl) => {
    return client
    .url(authUrl)
    .loginGoogle()
    .waitForVisible('#scope0 + button', 3000)
    .moveToObject('#scope0')
    .moveToObject('#submit_approve_access')
    .waitForVisible('#submit_approve_access content', 3000)
    .click('#submit_approve_access content')
    .waitForVisible('#code', 3000)
    .getValue('#code');
};

var seleniumDocker, getSeleniumServer, createWebdriverClient,
    client, newToken, results, getCredentials;
var authorizeSelenium = (authUrl) => {
    console.log('Authorizing: ' + authUrl);
    return importer.interpretAll([
        'selenium docker',
        'set up selenium server',
        'webdriver client',
        'decrypt password',
        'log in Google'
    ])
    .then(r => {
        results = r;
        seleniumDocker = r[0].runInNewContext();
        getSeleniumServer = r[1].runInNewContext({seleniumDocker});
        createWebdriverClient = r[2].runInNewContext();
        return getSeleniumServer('act-selenium')
    })
    .then(() => {
        client = createWebdriverClient('localhost', 4444);
        getCredentials = results[3].runInNewContext({client});
        results[4].runInNewContext({client, getCredentials});
        return approveSelenium(authUrl);
    })
    .catch(e => console.log(e))
};
if (typeof client != 'undefined'
    && typeof client.authorizeSelenium == 'undefined') {
    client.addCommand('authorizeSelenium', authorizeSelenium);
}
module.exports = authorizeSelenium;
authorizeSelenium;



ReferenceError: client is not defined